<!-- ---
title: Reinforcement Learning with Ignite
date: 2021-11-22
downloads: true
weight: 3
tags:
  - RL
  - intermediate
--- -->
# Reinforcement Learning with Ignite

In this tutorial we will implement a [policy gradient based algorithm](http://www.scholarpedia.org/article/Policy_gradient_methods) called [Reinforce](http://www.cs.toronto.edu/~tingwuwang/REINFORCE.pdf) and use it to solve OpenAI's [Cartpole problem](https://github.com/openai/gym/wiki/CartPole-v0) using PyTorch-Ignite.
<!--more--> 

## Prerequisite

The reader should be familiar with the basic concepts of Reinforcement Learning like state, action, environment, etc.

## The Cartpole Problem

We have to balance a Cartpole which is a pole-like structure attached to a cart. The cart is free to move across the frictionless surface. We can balance the cartpole by moving the cart left or right in 1D. Let's start by defining a few terms.

### State

There are 4 variables on which the environment depends: cart position and velocity, pole position and velocity.

### Action space

There are 2 possible actions that the agent can perform: left or right direction.

### Reward

For each instance of the cartpole not toppling down or going out of range, we have a reward of 1.

### When is it solved?

The problem is considered solved when the average reward is greater than `reward_threshold` defined for the environment.



## Required Dependencies 

In [ ]:
!pip install gymnasium pytorch-ignite

### On Colab

We need additional dependencies to render the environment on Google Colab.

In [ ]:
!apt-get install -y xvfb python-opengl
!pip install pyvirtualdisplay

## Imports

In [ ]:
from collections import deque
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from ignite.engine import Engine, Events

import gymnasium as gym
from gymnasium.wrappers import RecordVideo

import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

## Configurable Parameters

We will use there values later in the tutorial at appropriate places.

In [ ]:
seed_val = 543
gamma = 0.99
log_interval = 100
max_episodes = 1000000
render = True

## Setting up the Environment

Let's load our environment first.

In [ ]:
env = gym.make("CartPole-v0")

### On Colab

If on Google Colab, we need to follow a list of steps to render the output. First we initialize our screen size.

In [ ]:
display = Display(visible=0, size=(1400, 900))
display.start()

Below we have a utility function to enable video recording of the gym environment. To enable video, we have to wrap our environment in this function.

In [ ]:
def wrap_env(env):
    env = RecordVideo(env, './video')
    return env

env = wrap_env(env)

## Model

We are going to utilize the reinforce algorithm in which our agent will use episode samples from starting state to goal state directly from the environment. Our model has two linear layers with 4 in features and 2 out features for 4 state variables and 2 actions respectively. We also define an action buffer as `saved_log_probs` and a rewards one. We also have an intermediate ReLU layer through which the outputs of the 1st layer are passed to receive the score for each action taken. Finally, we return a list of probabilities for each of these actions.



In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

And then we initialize our model, optimizer, epsilon and timesteps.
> TimeStep is the object which contains information about a state like current observation, type of the step, reward, and discount. Given that some action is performed on some state, it gives the new state, type of the new step (or state), discount, and reward achieved.

In [ ]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()
timesteps = range(10000)

## Create Trainer

Ignite's [`Engine`](https://pytorch.org/ignite/concepts.html#engine) allows users to define a `process_function` to run one episode. We select an action from the policy, then take the action through `step()` and finally increment our reward. If the problem is solved, we terminate training and save the `timestep`.

> An episode is an instance of a game (or life of a game). If the game ends or life decreases, the episode ends. Step, on the other hand, is the time or some discrete value which increases monotonically in an episode. With each change in the state of the game, the value of step increases until the game ends.

In [ ]:
def run_single_timestep(engine, timestep):
    observation = engine.state.observation
    action = select_action(policy, observation)
    engine.state.observation, reward, done, _, _ = env.step(action)
    if render:
        env.render()

    policy.rewards.append(reward)
    engine.state.ep_reward += reward

    if done:
        engine.terminate_epoch()
        engine.state.timestep = timestep

trainer = Engine(run_single_timestep)

Next we need to select an action to take. After we get a list of probabilities, we create a categorical distribution over them and sample an action from that. This is then saved to the action buffer and the action to take is returned (left or right).

In [ ]:
def select_action(policy, observation):
    state = torch.from_numpy(observation).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()

We initialize a list to save policy loss and true returns of the rewards returned from the environment. Then we calculate the policy losses from the advantage (`-log_prob * reward`). Finally, we reset the gradients, perform backprop on the policy loss and reset the rewards and actions buffer.

In [ ]:
def finish_episode(policy, optimizer, gamma):
    R = 0
    policy_loss = []
    returns = deque()
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        returns.appendleft(R)
    
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)

    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()

    del policy.rewards[:]
    del policy.saved_log_probs[:]

## Attach handlers to run on specific events

We rename the start and end epoch events for easy understanding.

In [ ]:
EPISODE_STARTED = Events.EPOCH_STARTED
EPISODE_COMPLETED = Events.EPOCH_COMPLETED

Before training begins, we initialize the reward in `trainer`'s state.

In [ ]:
trainer.state.running_reward = 10

When an episode begins, we have to reset the environment's state.

In [ ]:
@trainer.on(EPISODE_STARTED)
def reset_environment_state():
    torch.manual_seed(seed_val + trainer.state.epoch)
    trainer.state.observation, _ = env.reset(seed=seed_val + trainer.state.epoch)
    trainer.state.ep_reward = 0

When an episode finishes, we update the running reward and perform backpropogation by calling `finish_episode()`.

In [ ]:
@trainer.on(EPISODE_COMPLETED)
def update_model():
    trainer.state.running_reward = 0.05 * trainer.state.ep_reward + (1 - 0.05) * trainer.state.running_reward
    finish_episode(policy, optimizer, gamma)

After that, every 100 (`log_interval`) episodes, we log the results.

In [ ]:
@trainer.on(EPISODE_COMPLETED(every=log_interval))
def log_episode():
    i_episode = trainer.state.epoch
    print(
        f"Episode {i_episode}\tLast reward: {trainer.state.ep_reward:.2f}"
        f"\tAverage length: {trainer.state.running_reward:.2f}"
    )

And finally, we check if our running reward has crossed the threshold so that we can stop training.

In [ ]:
@trainer.on(EPISODE_COMPLETED)
def should_finish_training():
    running_reward = trainer.state.running_reward
    if running_reward > env.spec.reward_threshold:
        print(
            f"Solved! Running reward is now {running_reward} and "
            f"the last episode runs to {trainer.state.timestep} time steps!"
        )
        trainer.should_terminate = True

## Run Trainer

In [ ]:
trainer.run(timesteps, max_epochs=max_episodes)

In [ ]:
env.close()

### On Colab

Finally, we can view our saved video.

In [ ]:
mp4list = glob.glob('video/*.mp4')

if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
else: 
    print("Could not find video")

That's it! We have successfully solved the Cartpole problem!